<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install yfinance
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [27]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [28]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BERGEPAINT',
       'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR',
       'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT',
       'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH',
       'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIPRULI', 'INDIGO', 'INDUSINDBK',
       'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM',
       'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN',
       'NATIONALUM', 'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL',
       'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS',
       'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'WIPRO', 'ZEEL'], dtype=object)

In [29]:
mypf['InPortfolio'].value_counts()

0    51
1    10
Name: InPortfolio, dtype: int64

In [30]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1060.0

In [31]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()

In [32]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJAJFINSV.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[**************

In [33]:
stock_prec_dev('^NSEI').tail(5)

[*********************100%***********************]  1 of 1 completed


,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,
2022-12-16,18269.0,^NSEI,4.75,0.67,0.38,18812.0,15294.0,84.57,15.43
2022-12-19,18420.0,^NSEI,5.56,0.64,0.35,18812.0,15294.0,88.87,11.13
2022-12-20,18385.0,^NSEI,5.31,0.62,0.29,18812.0,15294.0,87.87,12.13
2022-12-21,18199.0,^NSEI,4.19,0.60,0.18,18812.0,15294.0,82.58,17.42
2022-12-22,18129.0,^NSEI,3.75,0.56,0.11,18812.0,15294.0,80.57,19.43


In [34]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [35]:
# heavy upside
df_prec_dev[df_prec_dev['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-22,442.0,GODREJIND.NS,-6.31,-0.21,0.22,644.0,405.0,15.56,84.44,638.0,44.34
1,2022-12-22,449.0,ICICIPRULI.NS,-12.94,-1.36,-1.46,616.0,444.0,3.05,96.95,640.0,42.54
2,2022-12-22,694.0,MOTILALOFS.NS,-9.17,-0.92,0.43,971.0,654.0,12.60,87.40,930.0,34.01
3,2022-12-22,1232.0,SBILIFE.NS,1.63,0.41,-0.16,1332.0,1025.0,67.33,32.67,1607.0,30.44
4,2022-12-22,395.0,TATAMOTORS.NS,-9.04,-0.46,-0.48,525.0,372.0,14.87,85.13,495.0,25.32
5,2022-12-22,243.0,ZEEL.NS,-7.78,-0.38,-0.77,349.0,209.0,24.61,75.39,303.0,24.69
6,2022-12-22,6451.0,BAJFINANCE.NS,-5.30,-0.35,-0.79,7853.0,5284.0,45.43,54.57,8015.0,24.24
7,2022-12-22,891.0,ICICIBANK.NS,6.02,0.95,-0.22,953.0,654.0,79.13,20.87,1094.0,22.78
8,2022-12-22,527.0,JUBLFOOD.NS,-9.32,-1.03,-1.44,786.0,464.0,19.52,80.48,643.0,22.01
9,2022-12-22,78.0,IBREALEST.NS,-12.89,-1.11,0.00,174.0,60.0,16.14,83.86,95.0,21.79


In [36]:
# near 52W low
df_prec_dev[df_prec_dev['Discount%'] > 90].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
1,2022-12-22,449.0,ICICIPRULI.NS,-12.94,-1.36,-1.46,616.0,444.0,3.05,96.95,640.0,42.54
26,2022-12-22,1018.0,TECHM.NS,-11.94,-1.21,-0.67,1806.0,965.0,6.30,93.70,1139.0,11.89
34,2022-12-22,387.0,WIPRO.NS,-14.48,-1.32,-0.50,722.0,375.0,3.57,96.43,425.0,9.82


In [37]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev[df_prec_dev['Dev%_200'] < -10].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
34,2022-12-22,387.0,WIPRO.NS,-14.48,-1.32,-0.50,722.0,375.0,3.57,96.43,425.0,9.82
1,2022-12-22,449.0,ICICIPRULI.NS,-12.94,-1.36,-1.46,616.0,444.0,3.05,96.95,640.0,42.54
9,2022-12-22,78.0,IBREALEST.NS,-12.89,-1.11,0.00,174.0,60.0,16.14,83.86,95.0,21.79
26,2022-12-22,1018.0,TECHM.NS,-11.94,-1.21,-0.67,1806.0,965.0,6.30,93.70,1139.0,11.89
47,2022-12-22,4382.0,LTIM.NS,-11.72,-1.27,-1.27,7562.0,3824.0,14.93,85.07,4654.0,6.21
59,2022-12-22,635.0,IRCTC.NS,-11.70,-0.42,-1.54,903.0,571.0,19.25,80.75,571.0,-10.08


In [38]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev[df_prec_dev['Dev%_200'] > 10].sort_values(by = 'Dev%_200', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
54,2022-12-22,312.0,CANBK.NS,23.69,2.78,1.33,329.0,175.0,88.77,11.23,310.0,-0.64
19,2022-12-22,914.0,AXISBANK.NS,16.32,1.91,1.36,950.0,626.0,88.98,11.02,1060.0,15.97
60,2022-12-22,4402.0,BRITANNIA.NS,15.59,1.73,1.46,4526.0,3094.0,91.32,8.68,3935.0,-10.61
20,2022-12-22,129.0,FEDERALBNK.NS,12.98,1.75,0.00,138.0,81.0,83.86,16.14,149.0,15.50
58,2022-12-22,2122.0,LT.NS,11.99,1.53,1.12,2195.0,1472.0,89.94,10.06,2036.0,-4.05
15,2022-12-22,1178.0,INDUSINDBK.NS,10.92,1.51,0.68,1264.0,778.0,82.30,17.70,1393.0,18.25
51,2022-12-22,1021.0,SUNPHARMA.NS,10.10,0.76,0.10,1052.0,785.0,88.24,11.76,1056.0,3.43


In [39]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
1,2022-12-22,449.0,ICICIPRULI.NS,-12.94,-1.36,-1.46,616.0,444.0,3.05,96.95,640.0,42.54
2,2022-12-22,694.0,MOTILALOFS.NS,-9.17,-0.92,0.43,971.0,654.0,12.60,87.40,930.0,34.01
3,2022-12-22,1232.0,SBILIFE.NS,1.63,0.41,-0.16,1332.0,1025.0,67.33,32.67,1607.0,30.44
8,2022-12-22,527.0,JUBLFOOD.NS,-9.32,-1.03,-1.44,786.0,464.0,19.52,80.48,643.0,22.01
13,2022-12-22,575.0,HDFCLIFE.NS,1.54,0.35,0.53,674.0,504.0,41.59,58.41,681.0,18.43
21,2022-12-22,4129.0,NAUKRI.NS,-1.57,-0.14,0.72,5696.0,3375.0,32.48,67.52,4677.0,13.27
22,2022-12-22,1515.0,INFY.NS,-3.69,-0.19,-0.52,1940.0,1365.0,26.09,73.91,1715.0,13.20
25,2022-12-22,1086.0,MUTHOOTFIN.NS,-4.37,-0.35,0.09,1540.0,956.0,22.32,77.68,1217.0,12.06
26,2022-12-22,1018.0,TECHM.NS,-11.94,-1.21,-0.67,1806.0,965.0,6.30,93.70,1139.0,11.89
52,2022-12-22,3480.0,DIVISLAB.NS,-7.16,-1.04,-0.20,4678.0,3266.0,15.13,84.87,3560.0,2.30


In [40]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

1     21.254059
2     16.992256
3     15.208594
8     10.996752
13     9.208094
21     6.630027
22     6.595054
25     6.025481
26     5.940545
52     1.149138
Name: Upside%, dtype: float64

In [41]:
# prospects buy
tmp = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)
tmp[tmp['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-22,442.0,GODREJIND.NS,-6.31,-0.21,0.22,644.0,405.0,15.56,84.44,638.0,44.34
4,2022-12-22,395.0,TATAMOTORS.NS,-9.04,-0.46,-0.48,525.0,372.0,14.87,85.13,495.0,25.32
5,2022-12-22,243.0,ZEEL.NS,-7.78,-0.38,-0.77,349.0,209.0,24.61,75.39,303.0,24.69
6,2022-12-22,6451.0,BAJFINANCE.NS,-5.30,-0.35,-0.79,7853.0,5284.0,45.43,54.57,8015.0,24.24
7,2022-12-22,891.0,ICICIBANK.NS,6.02,0.95,-0.22,953.0,654.0,79.13,20.87,1094.0,22.78
9,2022-12-22,78.0,IBREALEST.NS,-12.89,-1.11,0.00,174.0,60.0,16.14,83.86,95.0,21.79
10,2022-12-22,590.0,SBIN.NS,9.62,1.49,0.17,626.0,435.0,81.02,18.98,710.0,20.34
11,2022-12-22,2467.0,TITAN.NS,-0.73,0.20,-0.81,2770.0,1936.0,63.66,36.34,2967.0,20.27


In [42]:
# CCI buy
tmp = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)
tmp[tmp['Upside%'] > 20].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
8,2022-12-22,527.0,JUBLFOOD.NS,-9.32,-1.03,-1.44,786.0,464.0,19.52,80.48,643.0,22.01
11,2022-12-22,2467.0,TITAN.NS,-0.73,0.20,-0.81,2770.0,1936.0,63.66,36.34,2967.0,20.27


In [43]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_50', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
39,2022-12-22,1995.0,INDIGO.NS,6.57,0.85,1.21,2265.0,1549.0,62.28,37.72,2176.0,9.07
37,2022-12-22,108.0,TATASTEEL.NS,-0.19,0.93,0.93,137.0,84.0,44.91,55.09,118.0,9.26
45,2022-12-22,896.0,GODREJCP.NS,4.19,0.47,0.69,968.0,665.0,76.24,23.76,958.0,6.92
46,2022-12-22,2662.0,HINDUNILVR.NS,7.23,0.85,0.57,2722.0,1944.0,92.34,7.66,2832.0,6.39
13,2022-12-22,575.0,HDFCLIFE.NS,1.54,0.35,0.53,674.0,504.0,41.59,58.41,681.0,18.43
41,2022-12-22,6895.0,ULTRACEMCO.NS,3.84,0.69,0.42,7870.0,5177.0,63.81,36.19,7420.0,7.61
40,2022-12-22,575.0,DABUR.NS,3.28,0.54,0.35,604.0,485.0,75.84,24.16,625.0,8.70
55,2022-12-22,892.0,MCDOWELL-N.NS,3.78,0.70,0.34,947.0,732.0,74.19,25.81,885.0,-0.78
38,2022-12-22,798.0,TATACONSUM.NS,3.07,0.52,0.25,849.0,660.0,72.91,27.09,871.0,9.15
48,2022-12-22,2742.0,HEROMOTOCO.NS,3.27,0.41,0.22,2913.0,2199.0,76.04,23.96,2903.0,5.87


In [44]:
# turning around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > 0)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-22,442.0,GODREJIND.NS,-6.31,-0.21,0.22,644.0,405.0,15.56,84.44,638.0,44.34
2,2022-12-22,694.0,MOTILALOFS.NS,-9.17,-0.92,0.43,971.0,654.0,12.60,87.40,930.0,34.01
21,2022-12-22,4129.0,NAUKRI.NS,-1.57,-0.14,0.72,5696.0,3375.0,32.48,67.52,4677.0,13.27
25,2022-12-22,1086.0,MUTHOOTFIN.NS,-4.37,-0.35,0.09,1540.0,956.0,22.32,77.68,1217.0,12.06


In [45]:
# bull run
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_50', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
39,2022-12-22,1995.0,INDIGO.NS,6.57,0.85,1.21,2265.0,1549.0,62.28,37.72,2176.0,9.07
45,2022-12-22,896.0,GODREJCP.NS,4.19,0.47,0.69,968.0,665.0,76.24,23.76,958.0,6.92
13,2022-12-22,575.0,HDFCLIFE.NS,1.54,0.35,0.53,674.0,504.0,41.59,58.41,681.0,18.43


In [46]:
# about to turn around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_50', ascending=False)

,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
21,2022-12-22,4129.0,NAUKRI.NS,-1.57,-0.14,0.72,5696.0,3375.0,32.48,67.52,4677.0,13.27
2,2022-12-22,694.0,MOTILALOFS.NS,-9.17,-0.92,0.43,971.0,654.0,12.60,87.40,930.0,34.01
0,2022-12-22,442.0,GODREJIND.NS,-6.31,-0.21,0.22,644.0,405.0,15.56,84.44,638.0,44.34
25,2022-12-22,1086.0,MUTHOOTFIN.NS,-4.37,-0.35,0.09,1540.0,956.0,22.32,77.68,1217.0,12.06
9,2022-12-22,78.0,IBREALEST.NS,-12.89,-1.11,0.00,174.0,60.0,16.14,83.86,95.0,21.79
14,2022-12-22,76.0,NATIONALUM.NS,-8.80,-1.20,0.00,130.0,67.0,13.81,86.19,90.0,18.42
52,2022-12-22,3480.0,DIVISLAB.NS,-7.16,-1.04,-0.20,4678.0,3266.0,15.13,84.87,3560.0,2.30
34,2022-12-22,387.0,WIPRO.NS,-14.48,-1.32,-0.50,722.0,375.0,3.57,96.43,425.0,9.82
26,2022-12-22,1018.0,TECHM.NS,-11.94,-1.21,-0.67,1806.0,965.0,6.30,93.70,1139.0,11.89
